In [1]:
cd ..

/Users/linafaik/Documents/projects/multiobj-optimization-pymoo


In [2]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go
from tqdm import tqdm

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.termination import get_termination

from src import (
    optimization,
    mcdm,
    visualization
)

%load_ext autoreload
%autoreload 2

## 1. Data Preprocessing

In [3]:
df = pd.read_csv("data/retail_store_inventory.csv")

In [4]:
column_demand = "Demand Forecast"
column_stock = "Available Units"
column_dispatch = "Inventory Level"
column_sales = "Unit Sold"

In [ ]:
# We assume that the total available stock in the warehouse on a given date
# is the sum of inventory levels across all stores. This is a basic assumption:
# no lead time, and no storage capacity limitations in stores.

# Ensure that demand values are non-negative (clip any negative values to 0)
df[column_demand] = df[column_demand].clip(0)

# Aggregate demand and dispatch quantities by Date and Product ID
available_stock_df = (
    df
    .groupby(["Date", "Product ID"], as_index=False)
    .agg({
        column_dispatch: "sum",   # Sum of dispatches = initial stock across all stores
        column_demand: "sum"      # Total demand across all stores
    })
)

# Rename the dispatch column to "Initial Stock" for clarity
available_stock_df = available_stock_df.rename(columns={column_dispatch: "Initial Stock"})

# Show the first few rows of the resulting DataFrame
available_stock_df.head()


,Date,Product ID,Initial Stock,Demand Forecast
0,2022-01-01,P0001,1403,779.23
1,2022-01-01,P0002,1569,759.79
2,2022-01-01,P0003,1752,754.32
3,2022-01-01,P0004,1686,451.78
4,2022-01-01,P0005,1462,726.28


In [ ]:
# We simulate shortage cases by introducing variability between demand and stock

np.random.seed(42)  # Set a random seed for reproducibility

# Define the range of random buffer units to simulate over- or under-stock
buffer_units_min = -200
buffer_units_max = 200

# Generate the simulated available stock:
# Subtract a random buffer (positive or negative) from demand to create stock values
# Clip negative values to 0 and round to whole units
available_stock_df[column_stock] = (
    available_stock_df[column_demand]
    - np.random.randint(buffer_units_min, buffer_units_max, size=len(available_stock_df))
).clip(lower=0).round()

# Identify shortage cases where demand exceeds available stock
shortage_cases_df = available_stock_df[
    (available_stock_df[column_demand]) > (available_stock_df[column_stock])
].reset_index(drop=True)

# Compute shortage quantity per case
shortage_cases_df["Shortage Units"] = (
    shortage_cases_df[column_demand] - shortage_cases_df[column_stock]
).clip(0)

# Show the first few shortage cases
shortage_cases_df.head()


,Date,Product ID,Initial Stock,Demand Forecast,Available Units,Shortage Units
0,2022-01-01,P0002,1569,759.79,612.0,147.79
1,2022-01-01,P0003,1752,754.32,684.0,70.32
2,2022-01-01,P0010,965,427.97,414.0,13.97
3,2022-01-01,P0011,1249,543.50,414.0,129.50
4,2022-01-01,P0013,1044,665.06,493.0,172.06


In [ ]:
import numpy as np
import plotly.graph_objects as go

i = 38 

# Extract relevant values
shortage_cases_df.sort_values(by="Shortage Units", ascending=False, inplace=True)
product_id = shortage_cases_df["Product ID"].iloc[i]
shortage_date = shortage_cases_df["Date"].iloc[i]

# Filter time series for the corresponding product
product_ts_df = available_stock_df[available_stock_df["Product ID"] == product_id].sort_values("Date")

# Find the index of the shortage date in the product's time series
shortage_idx = product_ts_df[product_ts_df["Date"] == shortage_date].index[0]
all_dates = product_ts_df["Date"].reset_index(drop=True)
date_position = all_dates[all_dates == shortage_date].index[0]

# Keep n_days dates before and after the shortage
n_days = 40
start = max(date_position - n_days, 0)
end = min(date_position + n_days + 1, len(product_ts_df))
filtered_ts_df = product_ts_df.reset_index(drop=True).iloc[start:end]

# Get shortage stock value
shortage_stock = filtered_ts_df.loc[filtered_ts_df["Date"] == shortage_date, column_stock].values[0]

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=filtered_ts_df["Date"],
    y=filtered_ts_df[column_stock],
    mode="lines+markers",
    name=column_stock,
    line=dict(color="#E9762B")  
))

fig.add_trace(go.Scatter(
    x=filtered_ts_df["Date"],
    y=filtered_ts_df[column_demand],
    mode="lines+markers",
    name=column_demand,
    line=dict(color="#3D8D7A")  
))

# Add red marker for shortage
fig.add_trace(go.Scatter(
    x=[shortage_date],
    y=[shortage_stock],
    mode="markers",
    marker=dict(color="red", size=10),
    name="Shortage"
))

fig.update_layout(
    title=f"Stock Time Series - Product ID {product_id}",
    xaxis_title="Date",
    yaxis_title=column_stock,
    template="plotly_white"
)

fig.show()


In [ ]:
# Merge the simulated stock levels back into the original dataframe
# We join on "Date" and "Product ID" to align each row with its corresponding stock level
# The merge is done as a left join to preserve all rows from the original 'df'

df = df.merge(
    available_stock_df[["Date", "Product ID", column_stock]],
    on=["Date", "Product ID"],
    how="left"
)


## 2. Multi-Objective Optimization with NSGA-II

In [25]:
shortage_to_optim_df = (
    df[
        (df["Date"] == shortage_cases_df["Date"].loc[i]) &
        (df["Product ID"] == shortage_cases_df["Product ID"].loc[i])
        ]
    .sort_values(by="Date")
    .reset_index(drop=True)
    )

sampling = FloatRandomSampling()           # Initial random population
crossover = SBX(prob=0.8, eta=15)          # Simulated Binary Crossover
mutation = PM(eta=20)                      # Polynomial Mutation

nsga2_algo = NSGA2(
    pop_size=100,                          # Population size
    sampling=sampling,                     # How to generate initial population
    crossover=crossover,                   # Crossover method
    mutation=mutation,                     # Mutation method
    eliminate_duplicates=True              # Avoid duplicate individuals
)

termination = get_termination("n_gen", 150)  # Run for 150 generations

# Run optimization for that day
problem = optimization.DispatchProblem(
    data=shortage_to_optim_df,
    column_demand=column_demand,
    column_stock=column_stock,
    with_demand_uncertainty=True,
    nb_scenario = 100,
    std_forecast_error = 0.35,
)
result = optimization.run_dispatch_optimization(
    problem=problem,
    algorithm=nsga2_algo,
    termination=termination,
    seed=123, # For reproducibility
    verbose=True # Show progress
)

# result shape = (Number of non-dominated solutions x number of stores)

# Best solution (vector of dispatch quantities, one per row/store)
best_idx = result.F[:, 0].argmin()
best_dispatch_vector = result.X[best_idx]

results = []

for row_idx, (store_id, forecast, stock, optimized_dispatch) in enumerate(zip(
    shortage_to_optim_df["Store ID"].values,
    shortage_to_optim_df[column_demand].values,
    shortage_to_optim_df[column_stock].values,
    best_dispatch_vector
)):
    optimized_sales = min(optimized_dispatch, forecast)

    results.append({
        "Date": shortage_to_optim_df["Date"].iloc[0],
        column_stock: stock,
        "Store ID": store_id,
        column_demand: forecast,
        "Dispatch": optimized_dispatch,
        "Sales": optimized_sales,
    })

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  6.420285E+01 |  1.698904E+03 |             - |             -
     2 |      200 |      1 |  2.664616E+01 |  9.194562E+02 |             - |             -
     3 |      300 |      2 |  0.000000E+00 |  4.664552E+02 |             - |             -
     4 |      400 |      4 |  0.000000E+00 |  1.337646E+02 |  0.7242311153 |         ideal
     5 |      500 |      5 |  0.000000E+00 |  5.5851895587 |  0.3244278516 |         ideal
     6 |      600 |      8 |  0.000000E+00 |  0.000000E+00 |  0.0717715152 |         ideal
     7 |      700 |      8 |  0.000000E+00 |  0.000000E+00 |  0.0601546790 |         ideal
     8 |      800 |      5 |  0.000000E+00 |  0.000000E+00 |  2.3297226262 |         nadir
     9 |      900 |      6 |  0.000000E+00 |  0.000000E+00 |  0.1654495296 |         ideal
    10 |     1000 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0545532189 |         ideal

In [10]:
# Create result DataFrame
results_df = pd.DataFrame(results)

# Show results
results_df

,Date,Available Units,Store ID,Demand Forecast,Dispatch,Sales
0,2022-01-04,1154.0,S001,349.59,338.977451,338.977451
1,2022-01-04,1154.0,S002,54.96,56.870043,54.960000
2,2022-01-04,1154.0,S003,248.10,240.329920,240.329920
3,2022-01-04,1154.0,S004,266.14,263.545550,263.545550
4,2022-01-04,1154.0,S005,252.35,246.797893,246.797893


In [11]:
result.F.shape  # → (n_solutions, 2)
# Each row in result.F is a non-dominated solution found by the algorithm
# Each column corresponds to one objective function

(25, 2)

In [12]:
result.F[0]
# = (f1, f2)
# where:
#   f1 = -sum(min(x, demand)) → maximize sales
#   f2 = sum(max(x - demand, 0)) → minimize overstock

array([-1115.09401737,    20.26644821])

In [13]:
result.X.shape # → (n_solutions, n_variables)
# Each row in result.X is a non-dominated solution found by the algorithm
# Each column corresponds to a decision variable — in dispatch case, the dispatch for one store to which you're dispatching a product on a given day.

(25, 5)

In [14]:
result.G.shape # → (n_solutions, n_constraints)
# Constraint violation values (should be <= 0 to be feasible)
# Each row corresponds to a solution; each column to a constraint

(25, 1)

In [15]:
result.cv 
# Constraint violation
# Used to identify feasible solutions: cv = 0 means fully feasible

array([0.])

In [16]:
result.feas
# result.feas is a boolean mask indicating which solutions in result.X are feasible.
# True = all constraints are satisfied (cv = 0), False = some constraint(s) violated

array([ True])

In [17]:
print("Execution time (sec):", result.exec_time)
# Total time taken to complete the optimization

print("End time:",pd.to_datetime(result.end_time, unit='s'))
# Timestamp when the optimization finished

Execution time (sec): 1.0391461849212646
End time: 2025-04-24 11:12:13.599990129


In [18]:
visualization.plot_objective_space(
    result, 
    y_label="Expected overstock", 
    x_label="Neg. expected sales", 
    title="Objective Space",
    show_ideal_nadir=True,
    scale_objectives=False
    ).update_layout(width=1000, height=500)

## 3. Multi-Criteria Decision Making

In [19]:
# Preference weights (higher = more important)
# Example: prioritize maximizing sales (minimizing -sales)
weights = [0.7, 0.3]  # f1 (sales), f2 (overstock)

In [20]:
# Using ASF
idx_asf, f_asf = mcdm.find_best_solution_by_asf(result, weights, scale=True)
print("Best (ASF):")
print("Index:", idx_asf)
print("Objectives:", f_asf)


Best (ASF):
Index: 5
Objectives: [-1089.99795595     8.52049958]


In [21]:
# Using PseudoWeights
idx_pw, f_pw = mcdm.find_best_solution_by_pseudo_weights(result, weights, scale=True)
print("Best (PseudoWeights):")
print("Index:", idx_pw)
print("Objectives:", f_pw)

Best (PseudoWeights):
Index: 1
Objectives: [-1105.19697939    12.85555905]


In [22]:
# Highlight both points in the same plot
visualization.plot_objective_space(
    result,
    x_label="-Expected Sales",
    y_label="Expected Overstock",
    title="Pareto Front - ASF vs PseudoWeights",
    scale_objectives=False,
    show_ideal_nadir=True,
    highlight_points=[
        {"index": idx_asf, "label": "ASF Optimal Point", "color": "red"},
        {"index": idx_pw, "label": "PseudoWeights  Optimal Point", "color": "orange"}
    ]
).update_layout(width=1000, height=500)

In [23]:
# Highlight both points in the same plot
visualization.plot_objective_space(
    result,
    x_label="-Expected Sales",
    y_label="Expected Overstock",
    title="Pareto Front - ASF vs PseudoWeights",
    scale_objectives=True,
    show_ideal_nadir=True,
    highlight_points=[
        {"index": idx_asf, "label": "ASF Optimal Point", "color": "red"},
        {"index": idx_pw, "label": "PseudoWeights  Optimal Point", "color": "orange"}
    ]
).update_layout(width=1000, height=500)